# 1. 특정 폴더에 패키지 설치
파이썬 패키지를 활용하기 위해서는 특정 폴더에 해당 패키지 파일을 설치하고,
해당 폴더를 사용하겠다고 선언해줘야한다.

In [1]:
# 특정 폴더에 패키지 파일 설치 예시

# 패키지 파일을 설치할 폴더
# 띄어 쓰기가 있는 경우, cmd 에서는 전체를 쌍따옴표(")로 감싼다.
mypath = "C:/Program Files/Python38/"

# 주피터 노트북에서는 powershell 에서 구동되므로 띄어쓰기 있는 부분만 쌍따옴표(")로 감싼다.
mypath = 'C:/"Program Files"/Python38/'

# ! : Terminal, CMD, Powershell 등에서 명령어를 실행한다.
# pip : 파이썬으로 작성된 패키지 소프트웨어를 설치/관리하는 패키지 관리 시스템. 파이썬 3.4 이후 버전은 pip를 기본적으로 포함함

# pip list는 현재 설치된 패키지를 확인할 수 있음
!pip list

# pip install 특정 폴더에 특정 패키지를 설치할 수 있음
# --target=$myapth 부분 없이 pip install schedule 을 하면 기본 폴더에 설치되지만, target으로 특정 폴더에 설치하고 활용하는 것을 권장함.
!pip install --target=$mypath schedule

# pip를 업그레이드 할 수 있음
!pip install --target=$mypath --upgrade pip

Package            Version
------------------ ---------
appdirs            1.4.4
backcall           0.2.0
beautifulsoup4     4.10.0
bs4                0.0.1
certifi            2021.5.30
charset-normalizer 2.0.6
colorama           0.4.4
cssselect          1.1.0
cycler             0.11.0
DateTime           4.3
debugpy            1.4.1
decorator          5.0.9
Deprecated         1.2.12
fake-useragent     0.1.11
feedparser         6.0.8
finance-datareader 0.9.31
html5lib           1.1
idna               3.2
importlib-metadata 4.8.1
ipykernel          6.1.0
ipython            7.26.0
ipython-genutils   0.2.0
jedi               0.18.0
jupyter-client     6.1.12
jupyter-core       4.7.1
kiwisolver         1.3.2
lxml               4.6.3
matplotlib         3.4.3
matplotlib-inline  0.1.2
multitasking       0.0.9
mysql              0.0.3
mysql-connector    2.2.9
mysqlclient        2.0.3
numpy              1.21.2
pandas             1.3.3
pandas-datareader  0.10.0
parse              1.19.0
parso     

  Using cached pip-21.3.1-py3-none-any.whl (1.7 MB)


In [1]:
# 슬랙 알람 울리기
import requests
from public_data_config import apisdata

def post_message(token, channel, text):
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    print(response)
 
myToken = apisdata['slack']['token']
channel = apisdata['slack']['channel']
 
# 메세지 보내기 테스트
# post_message(myToken,channel,"test_message")

In [2]:
import time
import threading
import schedule

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta # 특정 날짜 계산하는 패키지
import pandas as pd
import numpy as np
import yfinance as yf
import yahoo_fin.stock_info as si

# yahoo_fin.stock_info module 
# 시장 상태 확인

In [3]:
df = yf.download(tickers='DVN', period='1d', interval='1m')
df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-11-23 11:34:00-05:00,42.884998,42.884998,42.794998,42.799999,42.799999,18731
2021-11-23 11:35:00-05:00,42.794998,42.840000,42.794998,42.830002,42.830002,10830
2021-11-23 11:36:00-05:00,42.825001,42.910000,42.810001,42.900002,42.900002,21926
2021-11-23 11:37:00-05:00,42.930000,43.015701,42.930000,43.014999,43.014999,35822
2021-11-23 11:38:37-05:00,43.070000,43.070000,43.070000,43.070000,43.070000,0


In [4]:
market_dict = {
    'dow' : None,
    'sp500' : None,
    'nasdaq' : None
}

# 시장별 티커 리스트 가져오기
for market in market_dict:
    tickers = None

    # dow 지수 ticker list
    if market == 'dow':
        tickers = si.tickers_dow()
        market_dict[market] = tickers
        print('%s : %s'%(market, len(tickers)))

    # sp500 지수 ticker list
    elif market == 'sp500':
        tickers = si.tickers_sp500()
        market_dict[market] = tickers
        print('%s : %s'%(market, len(tickers)))

    # nasdaq 지수 ticker list
    elif market == 'nasdaq':
        tickers = si.tickers_nasdaq()
        market_dict[market] = tickers
        print('%s : %s'%(market, len(tickers)))


dow : 30
sp500 : 505
nasdaq : 5264


In [5]:
yf.Tickers(market_dict['dow']).history(period='1y')['Close']

[*********************100%***********************]  30 of 30 completed


,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-23,113.152443,215.551559,114.752251,211.529999,171.203506,257.640015,40.453770,86.496384,145.979996,53.510002,...,74.035332,208.363449,133.100449,135.355515,131.577133,330.303558,206.889145,57.618496,36.582863,148.622314
2020-11-24,114.464355,213.844376,119.002701,218.490005,173.280121,260.839996,41.396141,90.857780,151.490005,55.222248,...,73.906219,212.082260,133.666046,134.916550,133.297226,331.240082,208.399857,58.010712,37.103001,149.045746
2020-11-25,115.319084,213.698868,119.190521,217.610001,171.056595,246.820007,41.289272,87.551102,149.089996,53.978718,...,73.832443,212.092178,134.499603,135.277496,132.935608,328.460114,209.602463,57.790688,37.632763,149.508575
2020-11-27,115.875648,218.063843,119.200394,216.500000,171.497375,247.630005,41.483574,86.762436,147.130005,53.659092,...,73.648003,213.440872,133.219513,135.209213,131.391434,333.142700,209.711807,57.953320,37.565342,149.282089
2020-11-30,118.320580,215.376938,117.223442,210.710007,170.037872,245.800003,41.794460,82.838127,148.009995,51.344189,...,74.136787,212.290512,133.666046,135.462814,126.710045,331.565369,209.065781,57.790688,36.611759,150.453873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-17,153.490005,205.820007,179.610001,226.619995,202.979996,308.019989,56.759998,115.419998,157.330002,58.340000,...,82.599998,339.119995,171.740005,147.100006,156.419998,448.950012,205.059998,51.730000,47.810001,141.940002
2021-11-18,157.869995,204.020004,176.210007,227.250000,201.419998,302.989990,53.630001,114.449997,155.580002,57.980000,...,83.050003,341.269989,171.350006,147.119995,156.029999,449.470001,203.330002,51.240002,47.820000,143.160004
2021-11-19,160.550003,206.080002,173.539993,214.130005,200.440002,301.170013,53.250000,111.910004,154.000000,57.340000,...,80.699997,343.109985,174.880005,146.820007,152.529999,440.000000,200.860001,50.860001,46.980000,142.389999


In [6]:
# 기초 성과 지표 만들기

def create_basic_performance_indicators(market, market_ticker_list):

    # NOTE : 금융 공학에서의 1년 동안의 일수 = 252
    financial_engineering_days = 252

    i_dict = {}
    gagr = None
    mdd = None
    vol = None
    sharpe = None

    try:
        # Get tickers close price
        df = yf.Tickers(market_ticker_list).history(period='1y')['Close']
            
    except Exception as e:
        error_message = '%s error : %s'%(market, e)
        post_message(myToken, channel, error_message)

    for ticker in df.columns.tolist():
        if ticker not in i_dict:
            i_dict[ticker] = {}

        if ticker in i_dict:
            recent_date = df.index[-1]
            if recent_date not in i_dict[ticker]:
                i_dict[ticker][recent_date] = {
                    'gagr' : None,
                    'mdd' : None,
                    'vol' : None,
                    'sharpe' : None
                }

            if recent_date in i_dict[ticker]:
                ticker_df = df[ticker].reset_index()
                # 1. GAGR(Compound Annual Growth Rate, 연평균 복리 수익률)
                # 1.1. 수정된 종가 데이터로 일별 수익률 생성하기
                ticker_df['daily_rtn'] = ticker_df[ticker].pct_change()  # 퍼센트 변화율

                # 1.2. 일별 수익률로 전략 수익률 계산하기
                ticker_df['st_rtn'] = (1 + ticker_df['daily_rtn']).cumprod()  # 누적 곱 계산 함수 return cumulative product over a DataFrame or Series axis.

                # 1.3. GAGR(Compound Annual Growth Rate, 연평균 복리 수익률) 계산하기
                # NOTE : 단위 %
                gagr = (ticker_df[ticker_df['Date']==recent_date]['st_rtn'] ** (financial_engineering_days / len(ticker_df.index)) - 1) * 100

                # 2. MDD(Max Draw Down, 최대 낙폭)
                # 2.1. 관측 기간 최고점 가격 계산
                historical_max = ticker_df[ticker].cummax()  # 관측 기간 동안 누적 최댓값

                # 2.2 일별 감소비율 계산
                daily_drawdown = ticker_df[ticker] / historical_max - 1.0  # 현재 수정 종가에서 누적 최댓값 대비 낙폭률 계산

                # 2.3 일별 최대 하락률 계산
                historical_dd = daily_drawdown.cummin()  

                # 2.4 MDD 계산하기
                # NOTE : 단위 %
                mdd = - 1 * historical_dd.min() * 100

                # 3. Vol(Volatility, 변동성)
                # NOTE 단위 % 
                vol = np.std(ticker_df['daily_rtn']) * np.sqrt(financial_engineering_days) * 100

                # 4. sharpe(ex-post Sharpe ratio, 사후적 사프 비율)
                # NOTE : 샤프 지수는 위험성 대비 수익성 지표. 사후적 샤프 비율은 자산의 실현 수익률을 사용함
                # 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
                # 1 보다 크면 좋은 것으로 평가
                sharpe = np.mean(ticker_df['daily_rtn']) / np.std(ticker_df['daily_rtn']) * np.sqrt(financial_engineering_days)

                # dictionary 에 계산 값 입력하기
                i_dict[ticker][recent_date]['gagr'] = gagr
                i_dict[ticker][recent_date]['mdd'] = mdd
                i_dict[ticker][recent_date]['vol'] = vol
                i_dict[ticker][recent_date]['sharpe'] = sharpe

    complete_message = '%s Complete basic_performance_indicators : %s'%(market, len(list(i_dict.keys())))
    post_message(myToken,channel, complete_message)
    return i_dict

In [7]:
market = 'dow'
dow_basic_index = create_basic_performance_indicators(market, market_dict[market])

[*********************100%***********************]  30 of 30 completed
<Response [200]>


In [8]:
market = 'sp500'
sp500_basic_index = create_basic_performance_indicators(market, market_dict[market])
sp500_basic_index

[*********************100%***********************]  505 of 505 completed
<Response [200]>


{'A': {Timestamp('2021-11-23 00:00:00'): {'gagr': 252    45.681332
   Name: st_rtn, dtype: float64,
   'mdd': 17.372699149410266,
   'vol': 19.552036536183902,
   'sharpe': 2.0313707840670667}},
 'AAL': {Timestamp('2021-11-23 00:00:00'): {'gagr': 252    44.405057
   Name: st_rtn, dtype: float64,
   'mdd': 28.350115342145255,
   'vol': 46.989270409769524,
   'sharpe': 1.0176101623219214}},
 'AAP': {Timestamp('2021-11-23 00:00:00'): {'gagr': 252    61.380134
   Name: st_rtn, dtype: float64,
   'mdd': 16.267617414122405,
   'vol': 25.1068902624341,
   'sharpe': 2.041021503753054}},
 'AAPL': {Timestamp('2021-11-23 00:00:00'): {'gagr': 252    41.687594
   Name: st_rtn, dtype: float64,
   'mdd': 18.59886123660082,
   'vol': 24.632684954699997,
   'sharpe': 1.544153504783338}},
 'ABBV': {Timestamp('2021-11-23 00:00:00'): {'gagr': 252    22.165523
   Name: st_rtn, dtype: float64,
   'mdd': 11.905947414688722,
   'vol': 20.136479851137068,
   'sharpe': 1.1001450749632145}},
 'ABC': {Timestamp('

In [9]:
def create_basic_index_dataframe(basic_index_dict):
    ilist = []
    dict = {}
    for ticker in basic_index_dict:
        try:
            recent_date = list(basic_index_dict[ticker].keys())[0]
        except Exception as e:
            print(ticker, e)
            pass
        gagr = basic_index_dict[ticker][recent_date]['gagr']
        mdd = basic_index_dict[ticker][recent_date]['mdd']
        vol = basic_index_dict[ticker][recent_date]['vol']
        sharpe = basic_index_dict[ticker][recent_date]['sharpe']
        dict = {
            'ticker' : ticker,
            'recent_date' : recent_date,
            'gagr' : gagr,
            'mdd' : mdd,
            'vol' : vol,
            'sharpe' : sharpe
        }
        
        ilist.append(dict)
    df = pd.DataFrame(ilist)
    return df

In [10]:
sp500_df = create_basic_index_dataframe(sp500_basic_index)
sp500_df.tail()

,ticker,recent_date,gagr,mdd,vol,sharpe
500,YUM,2021-11-23,"252 21.955705 Name: st_rtn, dtype: float64",9.372827,17.742352,1.211887
501,ZBH,2021-11-23,"252 -10.308218 Name: st_rtn, dtype: float64",27.661448,24.040526,-0.333479
502,ZBRA,2021-11-23,"252 64.750876 Name: st_rtn, dtype: float64",16.503551,26.689071,2.011594
503,ZION,2021-11-23,"252 73.898887 Name: st_rtn, dtype: float64",20.856147,35.413118,1.746112
504,ZTS,2021-11-23,"252 36.314083 Name: st_rtn, dtype: float64",14.857320,19.643989,1.683040


In [11]:
candidate_list = sp500_df[(sp500_df['sharpe'] > 1) & (sp500_df['vol'] > 40)].sort_values(by='sharpe', ascending=False)['ticker'].tolist()
print(len(candidate_list))
candidate_list

27


['DVN',
 'NVDA',
 'F',
 'BBWI',
 'FANG',
 'MRO',
 'ALB',
 'GNRC',
 'PXD',
 'TSLA',
 'AMAT',
 'MRNA',
 'APA',
 'EOG',
 'MOS',
 'ETSY',
 'OXY',
 'MGM',
 'HES',
 'FCX',
 'ENPH',
 'UAA',
 'LRCX',
 'SLB',
 'PVH',
 'DXC',
 'AAL']

In [12]:
df = yf.download(tickers=candidate_list, period='1d', interval='1m')
df.head()

[*********************100%***********************]  27 of 27 completed

17 Failed downloads:
- BBWI: No data found for this date range, symbol may be delisted
- APA: No data found for this date range, symbol may be delisted
- ENPH: No data found for this date range, symbol may be delisted
- MGM: No data found for this date range, symbol may be delisted
- SLB: No data found for this date range, symbol may be delisted
- EOG: No data found for this date range, symbol may be delisted
- F: No data found for this date range, symbol may be delisted
- LRCX: No data found for this date range, symbol may be delisted
- MRNA: No data found for this date range, symbol may be delisted
- MRO: No data found for this date range, symbol may be delisted
- AMAT: No data found for this date range, symbol may be delisted
- OXY: No data found for this date range, symbol may be delisted
- GNRC: No data found for this date range, symbol may be delisted
- TSLA: No data found for this date range, symbol may be d

Adj Close                                           \
                                 AAL         ALB AMAT APA BBWI        DVN DXC   
Datetime                                                                        
2021-11-23 09:30:00-05:00  20.090000  282.484985  NaN NaN  NaN  42.640099 NaN   
2021-11-23 09:31:00-05:00  20.100000  281.980011  NaN NaN  NaN  42.855000 NaN   
2021-11-23 09:32:00-05:00  19.990000  281.810211  NaN NaN  NaN  42.730000 NaN   
2021-11-23 09:33:00-05:00  19.984600  280.065002  NaN NaN  NaN  42.974998 NaN   
2021-11-23 09:34:00-05:00  19.878201  281.299988  NaN NaN  NaN  43.180000 NaN   

                                                ...   Volume           \
                          ENPH EOG        ETSY  ...      MOS MRNA MRO   
Datetime                                        ...                     
2021-11-23 09:30:00-05:00  NaN NaN  283.850006  ...  60735.0  NaN NaN   
2021-11-23 09:31:00-05:00  NaN NaN  285.670013  ...  10499.0  NaN NaN   
2021-11-23 09:32:00-05:00  NaN NaN  287.260010  ...  13139.0  NaN NaN   
2021-11-23 09:33:00-05:00  NaN NaN  288.880005  ...   5475.0  NaN NaN   
2021-11-23 09:34:00-05:00  NaN NaN  289.209991  ...   7661.0  NaN NaN   

                                                                         
                                NVDA OXY PVH      PXD SLB TSLA      UAA  
Datetime                                                                 
2021-11-23 09:30:00-05:00  2599384.0 NaN NaN  35086.0 NaN  NaN  82106.0  
2021-11-23 09:31:00-05:00   504957.0 NaN NaN   6548.0 NaN  NaN  34042.0  
2021-11-23 09:32:00-05:00   467775.0 NaN NaN   8312.0 NaN  NaN  30006.0  
2021-11-23 09:33:00-05:00   429190.0 NaN NaN  21646.0 NaN  NaN  20306.0  
2021-11-23 09:34:00-05:00   455230.0 NaN NaN  17206.0 NaN  NaN  16817.0  

[5 rows x 162 columns]

In [13]:
def get_live_price(candidate_list):
    df = yf.download(tickers=candidate_list, period='1', interval='1m')
    close_column_name = 'Adj Close'
    adjclose_df = df[close_column_name]
    for ticker in adjclose_df.columns.tolist():
        ticker_df = adjclose_df[ticker].reset_index().rename(columns={ticker:close_column_name})
        ticker_df['recent_rtn'] = ticker_df[close_column_name].pct_change()
        ticker_df['st_rtn'] = ((1 + ticker_df['recent_rtn']).cumprod() - 1) * 100   # 누적 곱 계산 함수 return cumulative product over a DataFrame or Series axis.
        if ticker_df[ticker_df[close_column_name].isna() == False].iloc[-1]['st_rtn'] < -1:
            buy_message = 'Buy Now!! Ticker : %s, Recent Price : %s'%(ticker, ticker_df[ticker_df[close_column_name].isna() == False].iloc[-1][close_column_name])
            print(buy_message)
            post_message(myToken, channel, buy_message)
            pass

In [14]:
get_live_price(candidate_list)

[*********************100%***********************]  27 of 27 completed

18 Failed downloads:
- NVDA: No data found for this date range, symbol may be delisted
- UAA: No data found for this date range, symbol may be delisted
- ETSY: No data found for this date range, symbol may be delisted
- MGM: No data found for this date range, symbol may be delisted
- SLB: No data found for this date range, symbol may be delisted
- F: No data found for this date range, symbol may be delisted
- LRCX: No data found for this date range, symbol may be delisted
- MRO: No data found for this date range, symbol may be delisted
- AAL: No data found for this date range, symbol may be delisted
- MOS: No data found for this date range, symbol may be delisted
- ENPH: No data found for this date range, symbol may be delisted
- GNRC: No data found for this date range, symbol may be delisted
- FCX: No data found for this date range, symbol may be delisted
- OXY: No data found for this date range, symbol may be del

IndexError: single positional indexer is out-of-bounds

In [ ]:
# 일정 시간마다 특정 함수 수행 스케줄링
schedule.every(1).minutes.do(get_live_price, candidate_list)
# ticker = get_ticker()
# schedule.every(1).minutes.do(get_market_status, ticker)
# market_status, earnings_date = get_market_status(ticker)
# text = 'make_time : %s, market_status : %s, ticker : %s, next_earning_date : %s'%(datetime.now(), market_status, ticker, earnings_date)
# schedule.every(1).minutes.do(post_message,text)

# schedule.every().day.at("11:08").do(post_message,text)

while True:
    print('try : ', datetime.now())
    schedule.run_pending()
    time.sleep(60)
    

try :  2021-11-24 01:36:08.370792
try :  2021-11-24 01:37:08.375145
[*********************100%***********************]  26 of 26 completed
Buy Now!! Ticker : AAL, Recent Price : 19.579999923706055
[*********************100%***********************]  26 of 26 completed
Buy Now!! Ticker : AAL, Recent Price : 19.579999923706055


In [ ]:
#  특정 티커별 데이터 가져오기
def get_certain_ticker_data(certain_ticker):
    df = si.get_data(certain_ticker)
    
    # 1. 바이앤 홀드 전략
    # 수정된 종가 데이터 가져오기
    bh_df = df.loc[:, ['adjclose']].copy()

    # 수정된 종가 데이터로 일별 수익률 생성하기
    bh_df['daily_rtn'] = bh_df['adjclose'].pct_change()  # 퍼센트 변화율
    
    # 일별 수익률로 전략 수익률 계산하기
    bh_df['st_rtn'] = (1 + bh_df['daily_rtn']).cumprod()
    
    return bh_df

df = get_certain_ticker_data('AAPL')
df.head()

In [ ]:
# buy and hold strategy's returns arithmetic mean
base_date = '2021-11-01'
tmp_df = df.loc[base_date:,['st_rtn']] / df.loc[base_date, ['st_rtn']]
last_date = tmp_df.index[-1]
bh_ar_mean = round((tmp_df.loc[last_date, 'st_rtn'] - 1)*100, 3)

In [ ]:
# buy and hold strategy's 
bh_ge_mean = df.loc[base_date, 'st_rtn']

In [ ]:
round((tmp_df.loc[last_date, 'st_rtn'] - 1)*100, 3)

In [ ]:
tmp_df.loc[last_date, 'st_rtn']

In [ ]:
df.loc[base_date:,['st_rtn']] / df.loc[base_date, ['st_rtn']]

In [ ]:
df.loc[base_date, ['st_rtn']]

In [ ]:
price_df['daily_rtn'] = price_df['adjclose'].pct_change()
price_df['st_rtn'] = (1 + price_df['daily_rtn']).cumprod()
price_df.head(10)

In [ ]:
# 1. 평균 회귀 전략
n = 20
sigma = 2
def bollinger_band(price_df, n, sigma):
    bb = price_df.copy()
    bb['center'] = bb['adjclose'].rolling(n).mean()  # 중앙 이동 평균선
    bb['ub'] = bb['center'] + sigma * bb['adjclose'].rolling(n).std()  # 상단 밴드
    bb['lb'] = bb['center'] - sigma * bb['adjclose'].rolling(n).std()  # 하단 밴드
    return bb

bollinger = bollinger_band(price_df, n, sigma)
bollinger

In [ ]:
base_date = '2021-09-01'
sample = bollinger.loc[base_date:]
sample.head()

In [ ]:
book = sample[['adjclose']].copy()
book['trade'] = ''
book.head()

In [ ]:
def create_trade_book(sample):
    book = sample[['adjclose']].copy()
    book['trade'] = ''
    return book


In [ ]:
def tradings(sample, book):
    for i in sample.index:
        if sample.loc[i, 'adjclose'] > sample.loc[i, 'ub']:  # 상단 밴드 이탈 시 동작 안함
            book.loc[i, 'trade'] = ''
        elif sample.loc[i, 'lb'] > sample.loc[i, 'adjclose']:  # 하방 밴드 이탈 시 매수
            if book.shift(1).loc[i, 'trade'] == 'buy':  # 이미 매수 상태라면
                book.loc[i, 'trade'] = 'buy'  # 매수 상태 유지
            else:
                book.loc[i, 'trade'] = 'buy'
        elif sample.loc[i, 'ub'] >=  sample.loc[i, 'adjclose'] and sample.loc[i, 'adjclose'] >= sample.loc[i, 'lb']:  # 볼린저 밴드 안에 있을 시
            if book.shift(1).loc[i, 'trade'] == 'buy':
                book.loc[i, 'trade'] = 'buy'  # 매수 상태 유지
            else:
                book.loc[i, 'trade'] == ''  # 동작 안함
    return book


In [ ]:
book = tradings(sample, book)
book.tail(10)

In [ ]:
def returns(book):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        # long 진입
        if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':
            buy = book.loc[i, 'adjclose']
            print('진입일 : ', i, 'long 진입 가격 : ', buy)
        # long 청산
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':
            sell = book.loc[i, 'adjclose']
            rtn = (sell - buy) / buy + 1 #  손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일 : ', i, 'long 진입 가격 : ', buy, 'long 청산 가격 : ', sell, '| return : ', round(rtn, 4))
        if book.loc[i, 'trade'] == '':  # 제로 포지션
            buy = 0.0
            sell = 0.0
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, 'return']
        acc_rtn =  acc_rtn * rtn  # 누적 수익률 계산
        book.loc[i, 'acc_return']  = acc_rtn
    print('Accumulated return : ', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [ ]:
returns(book)

In [ ]:
temp_df = price_df.loc[base_date:, ['st_rtn']] / price_df.loc[base_date,['st_rtn']]
last_date = temp_df.index[-1]
temp_df.loc[last_date, 'st_rtn']

In [ ]:
last_date

In [ ]:
base_date

In [ ]:
range = (last_date - datetime.strptime(base_date, '%Y-%m-%d')).days
CAGR = price_df.loc[base_date, 'st_rtn'] ** (range/len(price_df.loc[base_date:].index)) - 1
CAGR

In [ ]:
print(base_date, last_date)

In [ ]:
import matplotlib.pylab as plt
book['acc_return'].plot()

In [ ]:
# 2. 듀얼 모멘텀 전략
t_df = pd.DataFrame()

In [ ]:
si.get_stats(dow_list[0])

In [ ]:
t_dict = si.get_analysts_info('AMZN')
analysts_list = list(t_dict.keys())
analysts_list

In [ ]:
i = 2
print(analysts_list[i])
t_dict[analysts_list[i]]

In [ ]:
t_dict['Earnings Estimate']

In [ ]:
# slack message 보내기
import requests
from public_data_config import apisdata

def post_message(text):
    token=apisdata['slack']['token']
    channel = apisdata['slack']['channel']
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    print(response, datetime.now())

def get_ticker():
    ticker = 'MSFT'
    return ticker

def get_market_status(ticker):
    market_status = si.get_market_status()
    earnings_date = si.get_next_earnings_date(ticker)
    return market_status, earnings_date

In [ ]:
# 일정 시간마다 특정 함수 수행 스케줄링
schedule.every(1).minutes.do(get_ticker)
ticker = get_ticker()
schedule.every(1).minutes.do(get_market_status, ticker)
market_status, earnings_date = get_market_status(ticker)
text = 'make_time : %s, market_status : %s, ticker : %s, next_earning_date : %s'%(datetime.now(), market_status, ticker, earnings_date)
schedule.every(1).minutes.do(post_message,text)

# schedule.every().day.at("11:08").do(post_message,text)

while True:
    print('try : ', datetime.now())
    schedule.run_pending()
    time.sleep(10)
    

In [ ]:
dow_list = si.tickers_dow() ## 30개
nasdaq_list = si.tickers_nasdaq() ## 5066개
sp500_list = si.tickers_sp500()  ## 505개

"""
dow_list 와 nasdaq_list 의 중복 ticker는 APPL, AMGN, CSCO, HON, INTC, MSFT, WBA
dow_list 에 들어있는 모든 30개사는 sp500에 포함되어 있음


"""


dow_list

In [ ]:
ticker_list = dow_list + nasdaq_list + sp500_list
ticker_list - list(set(ticker_list))

# 분석 시작일

date_to = datetime.now().date() - timedelta(days=3)

# 과거 데이터 확인 기간
past_data_years = 1
delta = relativedelta(years=past_data_years)

# 데이터 시작일
date_from = date_to - delta
str_date_from = date_from.strftime('%Y-%m-%d')
str_date_to = date_to.strftime('%Y-%m-%d')
print('데이터 시작일 : ', str_date_from, str_date_to)

# 시장 종류
# 'S&P500', 'NASDAQ', 'NYSE', 'AMEX', 'SSE', 'SZSE', 'HKEX', 'TSE', 'HOSE', 'KRX'
market = 'NASDAQ'

In [ ]:
# 시장별 ticker 가져오기
market_list = ['S&P500', 'NASDAQ', 'NYSE', 'KRX']

ticker_dict = dict()
market_df = pd.DataFrame()

for i in range(len(market_list)):
    
    market = market_list[i]
    df = pd.DataFrame()
    if i == 0:
        df = fdr.StockListing(market)
        df['Symbol'] = df['Symbol'].str.replace('/.', '')
        df['Market'] = market
        market_df = df
        print(market_list[i])
    else:
        df = fdr.StockListing(market)
        df['Symbol'] = df['Symbol'].str.replace('/.', '')
        df['Market'] = market
        market_df = market_df.append(df)
        print(market_list[i])
market_df.to_pickle()

In [ ]:
market_df.groupby('Market')['Symbol'].nunique()

In [ ]:
def make_inverstment_performance_indicator(df, target_price, date):
    daily_rtn = 'daily_rnt' # 일별 수익률
    st_rtn = 'st_rtn'  # 전략 수익률 수익률 누적의 곱

    df[daily_rtn] = df[target_price].pct_change()  # 일별 수익률
    df[st_rtn] = (1 + df[daily_rtn]).cumprod()  # 전략 수익률 = 수익률의 누적 곱
    
    # CAGR = Compound Annual Growth Rate 연평균 복리 수익률, 수익률이 높을 수록 좋다.
    CAGR = df.loc[date, st_rtn] ** (252./len(df.index)) -1  # 특정 시점까지의  최종 누적 수익률의 누적 연도 제곱근, 금융공학에서 1년은 252 영업일

    # MDD = Maximum Draw Down 최대 낙폭, MDD 값이 크면 변동성이 심한 주식
    historical_max = df[target_price].cummax()
    daily_drawdown = df[target_price] / historical_max - 1.0
    historical_dd = daily_drawdown.cummin()
    MDD = historical_dd.min()

    # VOL = Volatility 휘발성, 변동성
    VOL = np.std(df[daily_rtn]) * np.sqrt(252.)

    # Sharp = ex-post Sharp ratio, 사후적 사프 비율
    # 샤프지수와 같은 공식을 사용하지만, 자산의 실현 수익률을 사용한다는 점에서 다르다.
    # 샤프지수는 위험 대비 수익성 지표를 의미함
    # 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
    # 보통 1 이상이면 좋다고 판단함

    Sharp = np.mean(df[daily_rtn]) / VOL

    return CAGR, Sharp, VOL, MDD
    


In [ ]:
data = yf.Ticker('MSFT')
df = data.history(period='1d', start=str_date_from, end=str_date_to)['Close'].reset_index().set_index('Date')
df.head()

In [ ]:
df.index[-1]

In [ ]:
select_stock_dict = dict()

CAGR_criteria = 0.05 # 연평균 복리 수익률(Compound Annual Growth Rate)
Sharp_criteria = 0.01  # 샤프 지수는 0.01 이상 되면 좋다고 판단한다. (VOL을 곱하기 100 해주면 Sharp는 1)
VOL_criteria = 0.60  # 변동성 지수는 60 이상 -> 하루하루 변동성이 크면서 성장해야 단타를 자주 하지 않을까?
MDD_criteria = -0.90  # 최대 낙폭은 1년 이내에 50%까지 하락한 경우가 있으면 허용한다.
target_price = 'Close'  # 일별 종가를 기준으로 계산한다.

for ticker in ticker_dict[market_list[0]]:
    df = yf.Ticker(ticker).history(period='1y')
    # df = fdr.DataReader(ticker, str_date_from, str_date_to)[target_price].reset_index().set_index('Date')
    
    if date_to in pd.to_datetime(df.index.tolist()):
        CAGR, Sharp, VOL, MDD = make_inverstment_performance_indicator(df, target_price, str_date_to)
        # print(ticker, CAGR, Sharp, VOL, MDD)
        if CAGR >= CAGR_criteria and Sharp >= Sharp_criteria and VOL >= VOL_criteria and MDD >= MDD_criteria:
            if ticker not in select_stock_dict:
                select_stock_dict[ticker] = dict(
                    CAGR = None,
                    Sharp = None,
                    VOL = None,
                    MDD = None
                )
            if ticker in select_stock_dict:
                select_stock_dict[ticker] = dict(
                    CAGR = CAGR,
                    Sharp = Sharp,
                    VOL = VOL,
                    MDD = -1*MDD*100
                )
select_stock_dict

In [ ]:
fdr.DataReader('AAPL', '2020-01-01', '2020-01-03')

In [ ]:
import pandas as pd

url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'

df_listing = pd.read_html(url, header=0)[0]

cols_ren = {'회사명':'Name', '종목코드':'Symbol', '업종':'Sector', '주요제품':'Industry', 

                    '상장일':'ListingDate', '결산월':'SettleMonth',  '대표자명':'Representative', 

                    '홈페이지':'HomePage', '지역':'Region', }

df_listing = df_listing.rename(columns = cols_ren)

df_listing['Symbol'] = df_listing['Symbol'].apply(lambda x: '{:06d}'.format(x))

df_listing['ListingDate'] = pd.to_datetime(df_listing['ListingDate'])

df_listing

In [ ]:
ms_df = fdr.DataReader(nasdaq_ticker_df.iloc[0]['Symbol'], str_start_date, str_today_date)
ms_df.head()

In [ ]:
price_df = ms_df['Close'].reset_index().set_index('Date')

In [ ]:
from_date = '2020-01-01'
to_date = '2021-01-01'
price_df.loc[from_date:to_date].plot(figsize=(16,9))

In [ ]:
# 일별 수익률
price_df['daily_rtn'] = price_df['Close'].pct_change()
price_df.head()

In [ ]:
# 바이앤 홀드 전략의 누적 곱 계산
price_df['st_rtn'] = (1 + price_df['daily_rtn']).cumprod()
price_df.head()

In [ ]:
price_df['st_rtn'].plot(figsize=(16,9))

In [ ]:
base_date = '2020-04-01'
tmp_df = price_df.loc[base_date:,['st_rtn']] / price_df.loc[base_date,['st_rtn']]
last_date = tmp_df.index[-1]
print('누적 수익 : ', tmp_df.loc[last_date, 'st_rtn'])
tmp_df.plot(figsize=(16,9))

In [ ]:
### 투자 성과 분석 지표
# 1. 연평균 복리 수익률
# 기하 평균 N 개의 변수를 모두 곱한 값의 제곱근
# 가장 최근일의 최종 누적 수익률의 누적 연도 제곱근을 구한다. 금융공학에서 1년은 252 영업일로 계산
# ** 제곱
CAGR = price_df.loc[str_today_date, 'st_rtn'] ** (252/len(price_df.index)) - 1
CAGR

In [ ]:
# 2. 최대 낙폭(MDD Maximum Daily Drawdown)
historical_max = price_df['Close'].cummax()
daily_drawdown = price_df['Close'] / historical_max - 1.0
historical_dd = daily_drawdown.cummin()
historical_dd.plot()


In [ ]:
historical_dd[-1]

In [ ]:
# 3. 변동성
# 변동성은 금융 자산의 방향성에 대한 불확실성과 가격 등락에 대한 위험 예상 지표로 해석
# 수익률의 표준편차를 변동성으로 계산함
VOL = np.std(price_df['daily_rtn']) * np.sqrt(252.)
VOL

In [ ]:
# 4. 샤프 지수
# 샤프 지수는 위험 대비 수익성 지표라고 볼 수 있다.
# Ra 는 자산 수익률 Rb 는 무위험 수익률
# 사후적 샤프 비율 ex-post Sharpe ratio
# 공식은 같지만 자산의 실현 수익률을 사용한다는 점에서 샤프 지수와 다름
# 실현 수익률의 산술 평균 / 실현 수익률의 변동성으로 계산함
Sharpe = np.mean(price_df['daily_rtn']) / np.std(price_df['daily_rtn']) * np.sqrt(252.)
Sharpe

In [ ]:
import threading, time, signal
from datetime import timedelta

wait_time_seconds = 1

class ProgramKilled(Exception):
    pass

def foo():
    print(time.ctime())

def signal_handler(signum, frame):
    raise ProgramKilled

class Job(threading.Thread):
    def __init__(self, interval, execute, *args, **kwargs):
        threading.Thread.__init__(self)
        self.daemon = False
        self.stopped = threading.Event()
        self.interval = interval
        self.execute = execute
        self.args = args
        self.kwargs = kwargs
    
    def stop(self):
        self.stopped.set()
        self.join()

    def run(self):
        while not self.stopeed.wait(self.interval.total_seconds()):
            self.execute(*self.args, **self.kwargs)


if __name__ == "__main__":
    signal.signal(signal.SIGTERM, signal_handler)
    signal.signal(signal.SIGINT, signal_handler)
    job = Job(interval=timedelta(seconds=wait_time_seconds), execute=foo)
    job.start()

    while True:
        try:
            time.sleep(1)
        except ProgramKilled:
            print("Program killed: running cleanup code")
            job.stop()
            break

In [ ]:
import requests
from public_data_config import apisdata

def post_message(token, channel, text):
    response = requests.post("https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer "+token},
        data={"channel": channel,"text": text}
    )
    print(response)
 
myToken = apisdata['slack']['token']
channel = apisdata['slack']['channel']
 
post_message(myToken,channel,"test_message")